In [ ]:
'''
Nome: Marcos Paulo
Nome: Victor Manuel

Tema 6 - assunto futebol
'''

In [1]:
import json
import pickle
import nltk
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

nltk.download('punkt')
nltk.download('wordnet')

# Define the file paths
intents_file = 'intents.json'
words_file = 'words.pkl'
classes_file = 'classes.pkl'
x_file = 'x.pkl'
y_file = 'y.pkl'
model_file = 'model.h5'
pdf_file = 'lista de dados.pdf'

# Load and preprocess the data
intents = json.loads(open(intents_file).read())
lemmatizer = WordNetLemmatizer()

words = []
documents = []
classes = ['intencoes', 'oquedeseja', 'assuntoFutebol']
ignore_words = ["!", "@", "#", "$", "%", "*", "?"]

for intent_class in intents:
    for intent in intents[intent_class]:
        tag = intent['tag']
        classes.append(tag)

    for pattern in intent['patterns']:
        words.extend(nltk.word_tokenize(pattern.lower()))
        documents.append((nltk.word_tokenize(pattern.lower()), tag))

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Save the preprocessed data
pickle.dump(words, open(words_file, 'wb'))
pickle.dump(classes, open(classes_file, 'wb'))

training = []
output_empty = [0] * len(classes)

# Extract patterns from intents
training_data = []
for intent in intents.values():
    for item in intent:
        patterns = item['patterns']
        tag = item['tag']
        for pattern in patterns:
            training_data.append((pattern, tag))


for document in documents:
    bag = []
    pattern_words = document[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

x = list(training[:, 0])
y = list(training[:, 1])

pickle.dump(x, open(x_file, 'wb'))
pickle.dump(y, open(y_file, 'wb'))

# Define the neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))

# Compile and train the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(x), np.array(y), epochs=200, batch_size=5, verbose=1)
model.save(model_file)

# Generate a PDF file with data lists
def generate_pdf(data_lists):
    pdf_filename = pdf_file
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    x = 50
    y = 750

    c.setFont("Helvetica", 12)
    c.drawString(x, y, "Lista de Palavras (words):")
    y -= 20
    for word in data_lists["words"]:
        c.drawString(x, y, word)
        y -= 15

    y -= 20
    c.drawString(x, y, "Lista de Classes:")
    y -= 20
    for class_item in data_lists["classes"]:
        c.drawString(x, y, class_item)
        y -= 15

    y -= 20
    c.drawString(x, y, "Lista de Dados de Treino X:")
    y -= 20
    for x_item in data_lists["x"]:
        c.drawString(x, y, str(x_item))
        y -= 15

    y -= 20
    c.drawString(x, y, "Lista de Alvos de Treino Y:")
    y -= 20
    for y_item in data_lists["y"]:
        c.drawString(x, y, str(y_item))
        y -= 15

    c.save()

data_lists = {
    "words": words,
    "classes": classes,
    "x": x,
    "y": y,
}

generate_pdf(data_lists)

# Load the trained model
model = load_model(model_file)

# Define helper functions for text processing and class prediction
def tokenize_words(text):
    return nltk.word_tokenize(text.lower())

def bag_of_words(tokenized_words, words):
    bag = [0] * len(words)
    for w in tokenized_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return bag

def class_prediction(message, words, model):
    # Tokenize the input message
    tokenized_words = tokenize_words(message)
    # Convert tokenized words to input vector
    input_vector = [words.index(word) + 1 if word in words else 0 for word in tokenized_words]
    # Pad or truncate the input vector to match the expected input length
    input_vector = pad_or_truncate(input_vector, input_length=52)
    # Reshape the input vector to match the model's input shape
    input_vector = np.array(input_vector, dtype=np.float32).reshape(1, -1)
    # Predict the class probabilities for the input vector
    probabilities = model.predict(input_vector)[0]
    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(probabilities)
    # Get the corresponding class label
    predicted_class = classes[predicted_class_index]
    return predicted_class, probabilities

def pad_or_truncate(sequence, input_length):
    if len(sequence) < input_length:
        sequence = sequence + [0] * (input_length - len(sequence))
    else:
        sequence = sequence[:input_length]
    return sequence



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/200


C:\Users\Marcos\AppData\Local\Temp\ipykernel_2084\3784702080.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


4/4 [==============================] - 0s 3ms/step - loss: 2.9164 - accuracy: 0.3684
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 2.6393 - accuracy: 0.5263
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 2.1670 - accuracy: 0.7895
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.6772 - accuracy: 0.7368
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1901 - accuracy: 0.7895
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 0.9298 - accuracy: 0.7895
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 0.7363 - accuracy: 0.7895
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 0.5686 - accuracy: 0.7895
Epoch 9/200
4/4 [==============================] - 0s 997us/step - loss: 0.4918 - accuracy: 0.8421
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 0.5355 - accuracy: 0.7895
Epoch 11/200
4/4 [=====================

4/4 [==============================] - 0s 2ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0284 - accuracy: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 1ms/step - loss: 9.1452e-04 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 94/200
4/4 [===========

4/4 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 167/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 2ms/step - loss: 1.9087e-04 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 1000us/step - loss: 7.9315e-04 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0101 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0341 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 176/200

In [3]:
message = 'Oi'
predicted_class, probabilities = class_prediction(message, words, model)
print(predicted_class, probabilities)

1/1 [==============================] - 0s 14ms/step
saudacao [3.3196365e-31 5.7392696e-33 1.2580144e-29 5.5881337e-31 5.1674521e-30
 5.7830920e-33 7.6735862e-31 8.5434046e-17 1.0361485e-31 5.3197314e-34
 6.3323352e-22 1.3302641e-30 2.2730544e-31 5.2244339e-31 2.4328546e-29
 1.0942694e-32 3.3560409e-30 5.0729589e-30 2.7388672e-32 3.6959474e-35
 3.6697839e-30 1.0000000e+00]
